In [1]:
from collections import namedtuple

import itertools as itt
from tqdm import tnrange,tqdm_notebook

from ipywidgets import interact

from funcs import gps_coord, gln_coord, get_epoch_array, file_byte_iterator,  filter_epoch, general_parser, map_rec, Uchet

%pylab

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Category10
from scipy import signal
import scipy.io as io
from bokeh.transform import jitter
TOOLS = "pan,wheel_zoom,box_zoom,reset,box_select,lasso_select, save"
output_notebook()
#output_file('html/b1_{}.html'.format(filename[5:7]))

Loading BokehJS ...

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import FancyArrowPatch
import numpy as np
from mpl_toolkits.mplot3d import proj3d
from itertools import product, combinations

class Arrow3D(FancyArrowPatch):

    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0, 0), (0, 0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0], ys[0]), (xs[1], ys[1]))
        FancyArrowPatch.draw(self, renderer)

In [4]:
def get(filename, sv_arr):
    data = bytes(file_byte_iterator(filename))
    epoch_array = get_epoch_array(data)
    req_ids = [b'~~', b'SI', b'AN', b'rc', b'pc', b'RC', b'PC', b'GE', b'PV', b'EL', b'AZ']
    epoch_min_arr = [filter_epoch(req_ids, epoch) for epoch in epoch_array]

    pseudo_params = namedtuple('pseudo_params', ['usi','cp', 'pr', 'elev'])
    ephemerisGE = namedtuple('ephemerisGE', ['sv', 'tow', 'flags', 
                                 'iodc', 'toc', 'ura',
                                 'healthS', 'wn', 'tgd', 
                                 'af2', 'af1', 'af0', 'toe', 
                                 'iode', 'rootA', 'ecc', 'm0', 
                                 'omega0', 'inc0', 'argPer', 
                                 'deln', 'omegaDot', 'incDot', 
                                 'crc', 'crs', 'cuc', 'cus', 'cic', 'cis'])

    def pick_elem(dict):
        return [dict[b'SI'].usi, dict[b'PC'].cp, dict[b'RC'].pr, dict[b'EL'].elev]

    def process_epoch(epoch):
        ep = [map_rec(general_parser(rec)) for rec in epoch]
        ep_dict = {rec.id: rec for rec in ep}
        processed_data = {
            'RT': (ep_dict[b'~~'].tod[0]*1e-3)%86400,
            'pseudo_params': [pseudo_params(usi, cp, pr, elev) for usi, cp, pr, elev in zip( *pick_elem(ep_dict) )],
            'receiver': ep_dict[b'PV'],
        }
        if b'GE' in ep_dict:
            processed_data['ephemerisGE'] = ep_dict[ b'GE']
        return processed_data

    sats_array = [process_epoch(epoch) for epoch in epoch_min_arr]
    #sat_live = [[find_sat(sv, sats['pseudo_params']) for sv in sv_arr] for sats in sats_array]    
    
    return sats_array

fR = lambda xr,yr,zr: np.sqrt((xr)**2+(yr)**2+(zr)**2)

def sync(data, data1):
    epoch = 0
    X,Y = 0,0
    if data[0]['RT'] > data1[0]['RT']:
        while data[0]['RT'] != data1[epoch]['RT']:
            epoch += 1
        X,Y = data, data1[epoch:]
    else:
        while data1[0]['RT'] != data[epoch]['RT']:
            epoch += 1
        X,Y =  data[epoch:], data1
    if len(X) > len(Y):
        X = X[:len(Y)]
    else:
        Y = Y[:len(X)]
    return X,Y

In [5]:
pseudo_params = namedtuple('pseudo_params', ['usi','cp', 'pr', 'elev'])
def find_sat(usi, sats):
    return next((sat for sat in sats if (sat.usi == usi)), pseudo_params(usi, 0, 0, 0))
def popr(filename):
    if '1' in filename:
        return 0.169, 0.829, -0.452  
    elif '2' in filename:
        return -0.642, 1.169, -0.139  
    elif '3' in filename:
        return -0.839, 0.351, 0.293  

def select_eph(target, arr):
    compare = lambda x: abs(target - x)
    return sorted(arr, key=compare)[0]

In [6]:
def StatusGPS(sys, sys0,x0 = 0, y0 = 0, z0 = 0, rec_mode = 0):
    Status = []
    for epoch in tnrange(len(sys0)):
        stat = []
        #rt = int((sys[epoch]['RT'].tod[0]*1e-3)%86400)
        rt = sys[epoch]['RT']
        pv = B1[0]['receiver']
        sats = sys0[epoch]#['pseudo_params']
        for usi in sv_gps:
            if sats[usi].usi in EphemerisGE.keys():
                cur_eph = EphemerisGE[sats[usi].usi]
                X,Y,Z = Uchet(*gps_coord(rt, cur_eph, sats[usi].pr), pv.x+x0,pv.y+y0,pv.z+z0)
                R = fR( X-(pv.x+x0), Y-(pv.y+y0), Z-(pv.z+z0) )
                stat.append(status(rt, sats[usi].usi, sats[usi].usi, X,Y,Z,pv.x,pv.y,pv.z, R,(sats[usi].cp),sats[usi].pr, sats[usi].elev))
            else:
                stat.append(status(rt, sats[usi].usi, sats[usi].usi, 0,0,0,0,0,0, 0,(sats[usi].cp),sats[usi].pr, 0))
        Status.append({rec.usi: rec for rec in stat if rec.X != 0})
    return Status

In [7]:
status = namedtuple('status',['rt','usi', 'sv' ,'X', 'Y', 'Z', 'x', 'y', 'z' ,'R','cp', 'pr', 'el'])
status0 = namedtuple('status',['rt', 'usi', 'cp', 'R'])

d_x0 = namedtuple('d_x',['d_fi','d_R', 'd_p'])
dd_x0 = namedtuple('dd_x',['dd_fi','dd_R', 'dd_p'])

In [8]:
sv_gps = [*range(1,39)]
sv_arr = [4, 5, 6, 24, 25, 30]
F = 1575.42e6
C = 299792458.
lam = C / F

In [9]:
%%time
filename = 'data/r1.dat'
B1,R1 = sync(get('data/b1.dat', sv_gps), get(filename, sv_gps))

epge = [sat['ephemerisGE'] for sat in B1 if 'ephemerisGE' in sat]

EphemerisGE = {rec.sv: rec for rec in [sat['ephemerisGE'] for sat in R1 if 'ephemerisGE' in sat]}

Wall time: 6.28 s


In [10]:
dx,dy,dz = popr(filename)

bgps = {epoch: {usi: find_sat(usi, B1[epoch]['pseudo_params']) for usi in sv_gps} for epoch in range(len(B1))}
rgps = {epoch: {usi: find_sat(usi, R1[epoch]['pseudo_params']) for usi in sv_gps} for epoch in range(len(B1))}

In [11]:
bgps0 = {usi: {epoch: find_sat(usi, B1[epoch]['pseudo_params']) for epoch in range(len(B1))} for usi in sv_gps}

In [12]:
%%time
Bgps = StatusGPS(B1, bgps)
Rgps = StatusGPS(R1, rgps)#, dx, dy, dz)


Wall time: 44.4 s


In [13]:
# q = figure(width = 1200, plot_height = 250)
# for usi, color in zip(sv_gps0, Category10[10]):
#     q.line(Range, [bgps0[usi][epoch].cp for epoch in Range],
#            color = color, alpha = 0.8, muted_color=color, muted_alpha=0.2, legend=str(usi), line_width = 1)
# show(q)

In [14]:
d_x = namedtuple('d_x',['d_fi','d_qfi', 'd_R', 'd_p', ])
dd_x = namedtuple('dd_x',['dd_fi', 'dd_qfi', 'dd_R', 'dd_p',])
Y_x = namedtuple('Y_x', ['Yp', 'Yqfi', 'fi'])
h_x = namedtuple('h', ['hx', 'hy', 'hz'])
Hy_x = namedtuple('Hy_x', ['Hyp', 'Hyqfi', 'Hfi'])

In [15]:
sv_gps0 = [1, 4, 5, 6, 9,10, 14, 24, 25, 30]
sv_gps0 = [ 5, 6, 24, 25, 30]
JGP = len(sv_gps0)
ts = 0
step = 60*60
te = 4*60
Range = range(ts, te+1)

Index = lambda epoch: next(key for key, val in Bgps[epoch].items() if val.el == np.max([val.el for key, val in Bgps[epoch].items()]))

dX = {epoch: {sv: d_x(Rgps[epoch][sv].cp - Bgps[epoch][sv].cp,
                     (Rgps[epoch][sv].cp - Rgps[ts][sv].cp) - (Bgps[epoch][sv].cp - Bgps[ts][sv].cp),
                      Rgps[epoch][sv].R - Bgps[epoch][sv].R,
                      (Rgps[epoch][sv].pr - Bgps[epoch][sv].pr)*C,) for sv in sv_gps0} for epoch in Range }

el = 30
ddX = {epoch: {sv: dd_x(dX[epoch][sv].d_fi - dX[epoch][el].d_fi,
                       (dX[epoch][sv].d_qfi - dX[epoch][el].d_qfi)*lam,
                        dX[epoch][sv].d_R - dX[epoch][el].d_R,
                        dX[epoch][sv].d_p - dX[epoch][el].d_p,) for sv in sv_gps0} for epoch in Range }

ddM = {epoch: {sv: ddX[epoch][sv].dd_fi - ddX[epoch][sv].dd_R / lam for sv in sv_gps0} for epoch in Range }
ddMsv = {sv: [ddM[epoch][sv] - ddM[ts][sv] for epoch in Range] for sv in sv_gps0}

In [16]:
# p = figure(width = 1000, plot_height = 400,  title="Неопределенное целое b1-"+filename[5:7]+' для ГЛОНАСС', 
#            x_axis_label='Эпоха', y_axis_label='М',
#            tools=TOOLS, toolbar_location="below",)

# for usi, color in zip(sv_gps0, Category10[10]):
#     if usi == el: continue
#     p.line(Range, ddMsv[usi],
#            color = color, alpha = 0.8, muted_color=color, muted_alpha=0.2, legend=str(usi), line_width = 1)
# p.legend.click_policy="mute"
# p.legend.orientation = "horizontal"
# #p.xaxis.bounds = (1300, 2100)
# show(p)

In [17]:
# {epoch: {sv: for sv in sv_gps0} for epoch in Range}
# np.matrix([ (  ) for sv in sv_gps0 if sv != el])

In [18]:
Yp0 = {epoch: np.matrix([ ( ddX[epoch][sv].dd_R - ddX[epoch][sv].dd_p ) for sv in sv_gps0 if sv != el]).T for epoch in Range}
Yqfi0={epoch: np.matrix([ ( dX[epoch][sv].d_R - dX[ts][sv].d_R - (dX[epoch][el].d_R - dX[ts][el].d_R) - ddX[epoch][sv].dd_qfi )  for sv in sv_gps0 if sv != el]).T for epoch in Range}
Yfi0 ={epoch: np.matrix([ ( ddX[epoch][sv].dd_R / lam - ddX[epoch][sv].dd_fi ) for sv in sv_gps0 if sv != el]).T for epoch in Range}

In [19]:
# 15.12 Постваивл вместо Rgps - Bgps
dh = {epoch: {sv: h_x((Bgps[epoch][sv].X - Bgps[epoch][sv].x) / Bgps[epoch][sv].R,
                      (Bgps[epoch][sv].Y - Bgps[epoch][sv].y) / Bgps[epoch][sv].R,
                      (Bgps[epoch][sv].Z - Bgps[epoch][sv].z) / Bgps[epoch][sv].R,) for sv in sv_gps0} for epoch in Range}

In [20]:
HYp0 = {epoch: np.matrix([(dh[epoch][sv].hx - dh[epoch][el].hx,
                           dh[epoch][sv].hy - dh[epoch][el].hy,
                           dh[epoch][sv].hz - dh[epoch][el].hz) for sv in sv_gps0 if sv != el]) for epoch in Range}

In [21]:
HYqfi0 = {epoch: np.matrix([*zip([dh[epoch][sv].hx - dh[ts][sv].hx - (dh[epoch][el].hx - dh[ts][el].hx) for sv in sv_gps0 if sv != el],
                                 [dh[epoch][sv].hy - dh[ts][sv].hy - (dh[epoch][el].hy - dh[ts][el].hy) for sv in sv_gps0 if sv != el],
                                 [dh[epoch][sv].hz - dh[ts][sv].hz - (dh[epoch][el].hz - dh[ts][el].hz) for sv in sv_gps0 if sv != el])]) for epoch in Range}

In [22]:
Hfi_GP0 = {epoch: HYp0[epoch] / lam    for epoch in Range}

In [23]:
Y_GP0 = {epoch: np.concatenate( (Yp0[epoch], Yqfi0[epoch]) , axis = 0) for epoch in Range}

In [24]:
H_GP0 = {epoch: np.concatenate( ( (HYp0[epoch], HYqfi0[epoch], Hfi_GP0[epoch]) ) , axis = 0) for epoch in Range}

In [25]:
ddx, ddy, ddz = Rgps[te][sv_gps0[0]].x + dx - Bgps[te][sv_gps0[0]].x, Rgps[te][sv_gps0[0]].y + dy - Bgps[te][sv_gps0[0]].y, Rgps[te][sv_gps0[0]].z + dz - Bgps[te][sv_gps0[0]].z

dT = np.matrix([ddx, ddy, ddz]).T

In [26]:
us = 0
d = figure(width = 1200, plot_height = 250)
# d.line(Range, [np.array(np.abs( Yqfi0[te] - HYqfi0[te] * (dT) ))[0][0] for te in Range], color = 'red', legend = '5')
# d.line(Range, [np.array(np.abs( Yqfi0[te] - HYqfi0[te] * (dT) ))[1][0] for te in Range], color = 'blue', legend = '6')
# d.line(Range, [np.array(np.abs( Yqfi0[te] - HYqfi0[te] * (dT) ))[2][0] for te in Range], color = 'green', legend = '24')
# d.line(Range, [np.array(np.abs( Yqfi0[te] - HYqfi0[te] * (dT) ))[3][0] for te in Range], color = 'orange', legend = '25')
# d.line(Range, [np.array(Yfi0[te] )[us][0] - np.array(Yfi0[ts] )[us][0] for te in Range], legend = 'fi_GP')
# d.line(Range, [np.array( Hfi_GP0[te] * (dT) )[us][0] - np.array( Hfi_GP0[ts] * (dT) )[us][0] for te in Range], color = 'red', legend = f'Hfi * dTetta   sv:{sv_gps0[us]}')
d.line(Range, [np.array(np.abs( Yfi0[te] - Hfi_GP0[te] * (dT)))[us][0] -np.array(np.abs( Yfi0[0] - Hfi_GP0[0] * (dT)))[us][0] for te in Range], legend = f'разнциа sv:{sv_gps0[us]}')
d.legend.orientation = "horizontal"
d.legend.location = "top_left"
show(d)

In [27]:
qqq

NameError: name 'qqq' is not defined

In [ ]:
Rp = np.matrix(np.eye((JGP))* (1.5**2))
Rfi = np.matrix(np.eye((JGP))* (0.05**2))
Rqfi = 2 * Rfi

R1p = 2 * Rp
R1fi = 2 * Rfi
R1qfi = 2 * Rqfi
RW1qfi = R1qfi * lam**2

TR2 = np.eye((JGP-1))
TR2 = np.concatenate((TR2, np.matrix(np.ones((JGP-1))*-1).T), axis = 1)

R2p = TR2 * R1p * TR2.T
R2fi = TR2 * R1fi * TR2.T
R2qfi = TR2 * RW1qfi * TR2.T
LAM = np.matrix(np.eye(JGP)*lam)
CRqfifi = 2 * TR2 * LAM * Rfi * TR2.T

Null = np.matrix(np.zeros((JGP-1, JGP-1)))

R_GP = np.concatenate((np.concatenate((R2p, Null, Null), axis = 0),
                np.concatenate((Null, R2qfi, CRqfifi.T), axis = 0),
                np.concatenate((Null, CRqfifi, R2fi), axis = 0)), axis = 1)

B = R_GP.I
D = B - B * H_GP0[te] * (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B

# D = np.arange(12*12).reshape((12,12))

Dpp = D[:(JGP-1)*2,:(JGP-1)*2]
Dqp = D[(JGP-1)*2:,:(JGP-1)*2]
Dpq = D[:(JGP-1)*2,(JGP-1)*2:]
Dqq = D[(JGP-1)*2:,(JGP-1)*2:]

In [ ]:
D = {te: (B - B * H_GP0[te] * (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B) for te in Range}
Dpp = {te: D[te][:(JGP-1)*2,:(JGP-1)*2] for te in Range}
Dqp = {te: D[te][(JGP-1)*2:,:(JGP-1)*2] for te in Range}
Dpq = {te: D[te][:(JGP-1)*2,(JGP-1)*2:] for te in Range}
Dqq = {te: D[te][(JGP-1)*2:,(JGP-1)*2:] for te in Range}

In [ ]:
kzv = {te: (-1) * Dqq[te].I * Dqp[te] * Y_GP0[te] - Yfi0[te] for te in Range}
KVF =lambda k, te: (k - kzv[te]).T * Dqq[te] * (k - kzv[te])

In [ ]:
QQ = np.array([Dqq[te] for te in range(10)])

In [ ]:
QQ[1:2,:,:,]

In [ ]:
scipy.io.savemat('test_mat.mat', mdict={'Dqq': np.array([Dqq[te] for te in range(10)]) }, oned_as = 'row')

In [ ]:
K = np.matrix(io.loadmat('MATtoPY')['kmin'], dtype = 'int')

In [ ]:
dictionary = {'hello':'world'}
np.save('my_file.npy', dictionary) 

In [ ]:
i = 0
j = i+1
kmin = K[:,i:i+1]
ksled = K[:,j:j+1]
KNTR = (KVF(ksled, te) / KVF(kmin, te)).tolist()[0][0]
dT = (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B * np.concatenate((Y_GP0[te], Yfi0[te] + kmin), axis = 0)
print(KNTR)
[*zip(dT.tolist(), popr(filename))]

In [ ]:
dT

In [ ]:
dTT = matrix([[ 0.44627778, 0.60296689,0.51170608,0.16772547,0.16893263,0.16907246,0.17065871,0.17002797,0.17242257],
        [-0.47661323, 0.71541481,0.57934524,0.83104538,0.82913198,0.8278772 ,0.83075517,0.8290111 ,0.83143359],
        [-2.07079151, -0.1305015 ,-0.51566911,-0.4487098 ,-0.44933085,-0.44597202,-0.45112906,-0.4494524 ,-0.44917252]])

In [ ]:
Ran = ['1min', '2min', '3min', '5min', '10min', '20min', '30min', '40min', '60min']

In [ ]:
w = figure(width = 1200, plot_height = 250, x_range = Ran)
w.line(Ran, dTT.tolist()[0])
w.line(Ran, dTT.tolist()[1], color = 'red')
w.line(Ran, dTT.tolist()[2], color = 'green')
show(w)

In [ ]:
# for i in range(10-1):
#     j = i+1
#     kmin = K[:,i:i+1]
#     ksled = K[:,j:j+1]

#     dT = (H_GP0[te].T * B * H_GP0[te]).I * H_GP0[te].T * B * np.concatenate((Y_GP0[te], Yfi0[te] + kmin), axis = 0)
#     KNTR = (KVF(ksled) / KVF(kmin)).tolist()[0][0]
#     print(KNTR)
# print(dT, '  ', popr(filename))

In [ ]:
# %%time
# Bgln = StatusGLN(B1, bgln)
# Rgln = StatusGLN(R1, rgln, dx, dy, dz)

# B = [*Bgln[0].keys()]
# B = [55, 57]
# B0 = [18, 3]
# B = sv_arr0
# B0 = [5, 21, 18, 3]

# Range = range(len(Rgln))
# #Range = range(1300,2100)
# dX0 = {epoch: {sv: d_x0(Rgln[epoch][sv].cp - Bgln[epoch][sv].cp,
#                         Rgln[epoch][sv].R  - Bgln[epoch][sv].R,
#                         Rgln[epoch][sv].pr*C - Bgln[epoch][sv].pr*C) for sv in B} for epoch in Range}

# el = 55
# ddX0 = {epoch: {sv: dd_x0(dX0[epoch][sv].d_fi - dX0[epoch][el].d_fi,
#                           dX0[epoch][sv].d_R / lam_gln[sv] - dX0[epoch][el].d_R / lam_gln[el],
#                           (1 / (len(B) * (C / (F_gln[sv] - F_gln[el])))) * np.sum([dX0[epoch][svm].d_R - dX0[epoch][svm].d_p for svm in B])) for sv in B if sv != el} for epoch in  Range}

# ddM0 = {epoch: {sv: ddX0[epoch][sv].dd_fi - ddX0[epoch][sv].dd_R + ddX0[epoch][sv].dd_p for sv in B if sv != el} for epoch in  Range}
# ddMsv0 = {sv: [ddM0[epoch][sv]-ddM0[Range[0]][sv] for epoch in Range] for sv in B if sv != el}